# Image gathering
The robot takes pictures in 5 degree increments to familiarize itself with its environment.

In [9]:
import cozmo
from cozmo.util import degrees
import cv2
import os
from PIL import Image as im 

Set up the folder in which to store the images.

In [10]:
# File path configuration
parent_dir = r"C:\Users\Dressler\Pictures"
path = os.path.join(parent_dir, "cozmo")

# Only needs to be run once
try:
    os.mkdir(path)
except FileExistsError:
    print("Directory already exists")

Directory already exists


Method to rescale image. Used to downscale the images for faster processing later on.

In [ ]:
def resize_image (image, width, height):
    #convert image to float array
    img_array = image.astype("float")
    #use cv2 to resize array
    #tested multiple types of interpolation, INTER_LINEAR seems to be best, but feel free to test
    resized_array = cv2.resize(img_array, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
    #turn array back to image and return 
    resized_image = im.fromarray(resized_array)
    return resized_image

Take the pictures, storing them in the specified directory

In [11]:
def take_pictures(robot: cozmo.robot.Robot):
  # Initial setup, generate repeatable head angle and lift height
  robot.say_text("Taking pictures").wait_for_completed()
  move_arms = robot.set_lift_height(0)
  move_arms.wait_for_completed()
  set_head = robot.set_head_angle((cozmo.robot.MAX_HEAD_ANGLE) / 3, in_parallel = True)
  set_head.wait_for_completed()
  # Enabling Cozmo Camera
  robot.camera.image_stream_enabled = True

  deg_step = 5
  
  # Saves picture of what Cozmo sees every 10 degrees.
  for d in range(0, 360, deg_step):
    fileName = "image" + str(d) + ".jpeg"
 
    robot.turn_in_place(degrees(deg_step)).wait_for_completed()
    
    latest_image = robot.world.latest_image
    # annotation seems to be important, we have had issues saving images if we don't annotate
    annotated = latest_image.annotate_image()

    if latest_image is not None:
      print("image = %s" % latest_image)
      converted = annotated.convert()
      file_path = os.path.join(path, fileName)
      resized = resize_image(converted, 160, 120)   #image is resized before saving
      resized.save(file_path, "JPEG", resolution=10)

In [ ]:
cozmo.run_program(take_pictures)